# TruBot testing

This notebook tests a conversation bot with vector-store context of TruEra website. 

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent.parent.resolve()))

# Uncomment to get more debugging printouts:
"""
import logging

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

"\nimport logging\n\nroot = logging.getLogger()\nroot.setLevel(logging.DEBUG)\n\nhandler = logging.StreamHandler(sys.stdout)\nhandler.setLevel(logging.DEBUG)\nformatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')\nhandler.setFormatter(formatter)\nroot.addHandler(handler)\n"

In [3]:
from trulens_eval.keys import *
from trulens_eval.examples.trubot import get_or_make_chain, get_answer
from trulens_eval.util import TP
from trulens_eval import Tru
from trulens_eval.tru_feedback import Huggingface
from trulens_eval.util import instrumented_classes, jsonify
from trulens_eval.util import Obj, ObjSerial, Method, FunctionOrMethod, all_objects, Class
import json
import pydantic

from pprint import PrettyPrinter
pp = PrettyPrinter()

Tru().reset_database()

huggingface api: 0requests [00:00, ?requests/s]

In [4]:
chain = get_or_make_chain(cid=None)

'Starting a new conversation with 0/default.'
✅ chain 0/default -> default.sqlite
✅ feedback def. feedback_definition_hash_10c1c19cf057fa13087cdae84daac173 -> default.sqlite
✅ feedback def. feedback_definition_hash_33108b3d0b3b80d4615efd821c22d4fc -> default.sqlite
✅ feedback def. feedback_definition_hash_f98aee91b52e9783fc41bec4e7589ca5 -> default.sqlite


In [5]:
list(chain.instrumented())

[(JSONPath(), ['chain']),
 (JSONPath().memory, ['memory']),
 (JSONPath().memory.llm, ['llm']),
 (JSONPath().memory.prompt, ['prompt']),
 (JSONPath().memory.chat_memory, ['chathistory']),
 (JSONPath().combine_docs_chain, ['chain']),
 (JSONPath().combine_docs_chain.llm_chain, ['chain']),
 (JSONPath().combine_docs_chain.llm_chain.prompt, ['prompt']),
 (JSONPath().combine_docs_chain.llm_chain.llm, ['llm']),
 (JSONPath().combine_docs_chain.document_prompt, ['prompt']),
 (JSONPath().question_generator, ['chain']),
 (JSONPath().question_generator.prompt, ['prompt']),
 (JSONPath().question_generator.llm, ['llm']),
 (JSONPath().retriever, ['retriever'])]

In [ ]:
proc = Tru().start_dashboard(force=True, _dev=Path.cwd().parent.parent)
thread = Tru().start_evaluator(restart=True)

In [ ]:
selectors = [0,1,3,4]
messages = ["Who is Shayak?", "Wer ist Shayak?", "Kim jest Shayak?", "¿Quién es Shayak?", "Was ist QII?", "Co jest QII?"]

selectors = selectors[2:3]
messages = messages[2:3]

def test_bot(selector, question):
    print(selector, question)
    chain = get_or_make_chain(cid=question + str(selector), selector=selector)
    answer = get_answer(chain=chain, question=question)
    return answer

results = []

for s in selectors:
    for m in messages:
        results.append(TP().promise(test_bot, selector=s, question=m))

TP().finish()